## Overview 

In this demo, we will show:

1. How to represent actions that are available to a person using an `ActionSet` 
2. How to provide a consumer who is denied a loan by a machine learning model a list of actionable changes to flip their prediction with a `Flipset`
3. How to verify that a model will provide recourse to all of its decision subjects using the `RecourseAuditor`




Our library provides tools for recourse reporting and verifcation.

We'll start by building a machine learning model for loan approval that we'll use for the demo. 
We'll use a processed version of the  `german` credit dataset from the [UCI Machine Learning repository](https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)). 
We'll predict the risk of repayment with a simple logistic regression model. 

In [4]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import recourse as rs
from IPython.core.display import display, HTML

# import data
url = 'https://raw.githubusercontent.com/ustunb/actionable-recourse/master/examples/paper/data/credit_processed.csv'
df = pd.read_csv(url)
y, X = df.iloc[:, 0], df.iloc[:, 1:]

# train a classifier
clf = LogisticRegression(max_iter = 1000)
clf.fit(X, y)
yhat = clf.predict(X)

In [13]:
pd.Series(clf.coef_[0], index=X.columns).to_frame('Coefficients')

,Coefficients
Married,0.370584
Single,0.576174
Age_lt_25,0.207627
Age_in_25_to_40,0.436330
Age_in_40_to_59,0.323354
Age_geq_60,0.010576
EducationLevel,0.002482
MaxBillAmountOverLast6Months,0.000121
MaxPaymentAmountOverLast6Months,0.000031
MonthsWithZeroBalanceOverLast6Months,0.063924


# Action Set

In [ ]:
# customize the set of actions
A = rs.ActionSet(X)  ## matrix of features. ActionSet will set bounds and step sizes by default

# specify immutable variables
A['Married'].actionable = False

# education level
A['EducationLevel'].step_direction = 1  ## force conditional immutability.
A['EducationLevel'].step_size = 1  ## set step-size to a custom value.
A['EducationLevel'].step_type = "absolute"  ## force conditional immutability.
A['EducationLevel'].bounds = (0, 3)

A['TotalMonthsOverdue'].step_size = 1  ## set step-size to a custom value.
A['TotalMonthsOverdue'].step_type = "absolute"  ## discretize on absolute values of feature rather than percentile values
A['TotalMonthsOverdue'].bounds = (0, 100)  ## set bounds to a custom value.

# can only specify properties for multiple variables using a list
A[['Age_lt_25', 'Age_in_25_to_40', 'Age_in_40_to_59', 'Age_geq_60']].actionable = False
# todo: add one-hot constraint

In [10]:
A.df[['name','variable_type', 'actionable', 'lb', 'ub']]

,name,variable_type,actionable,lb,ub
0,Married,<class 'bool'>,False,0.0,1.0
1,Single,<class 'bool'>,True,0.0,1.0
2,Age_lt_25,<class 'bool'>,False,0.0,1.0
3,Age_in_25_to_40,<class 'bool'>,False,0.0,1.0
4,Age_in_40_to_59,<class 'bool'>,False,0.0,1.0
5,Age_geq_60,<class 'bool'>,False,0.0,1.0
6,EducationLevel,<class 'int'>,True,0.0,3.0
7,MaxBillAmountOverLast6Months,<class 'int'>,True,0.0,11321.0
8,MaxPaymentAmountOverLast6Months,<class 'int'>,True,0.0,5480.0
9,MonthsWithZeroBalanceOverLast6Months,<class 'int'>,True,0.0,4.0


In [22]:
# Person #13 is denied a loan (bad luck)
x = X.values[[13]]
yhat = clf.predict(x)[0]
yhat

0.0

In [23]:
# Let's produce a list of actions that can change this person's predictions
fs = rs.Flipset(x, action_set = A, clf = clf)
fs.populate(enumeration_type = 'distinct_subsets', total_items = 5)
html_str = fs.to_html()
display(HTML(html_str))

obtained 5 items in 0.4 seconds
Index(['item', 'Features to Change', 'Current Value', 'Required Value'], dtype='object')


Features to Change,Current Value,to,Required Value
TotalMonthsOverdue,7.000000,→,3.000000
MaxPaymentAmountOverLast6Months,100.000000,→,110.000000
TotalMonthsOverdue,7.000000,→,3.000000
MostRecentBillAmount,2010.000000,→,1926.000000
TotalMonthsOverdue,7.000000,→,3.000000
MostRecentPaymentAmount,100.000000,→,105.000000
TotalMonthsOverdue,7.000000,→,3.000000
MaxBillAmountOverLast6Months,2060.000000,→,2166.000000
TotalMonthsOverdue,7.000000,→,3.000000


In [ ]:
# todo: fs.actions[0] should return actions
# if you perform any of these actions, you will flip your prediction
# show

If a flipset is empty that means this person has no actions.

In [ ]:
# Person for which flipset is empty
# To-Do find a person who has no recourse
# no recourse = #2020

# These are cases where people have no recourse
# They have no action to obtain a desired outcome
# We could provide them with principals reasons for the denial, but it would be misleading.


# Recourse Verification

## Basic Audits

In [ ]:
# Basic Recourse Verification with 1 Model
# Use the auditor on 100 points (live)
# It's super easy

# How many people are dnied
# How many have recourse?
# How difficult is that recourse?

## Internal Audits for Model Development

In [ ]:
# Show normal graphs 2 x 1

In [ ]:
# Show normal graphs + recourse graphs (2 x 2)